In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_output

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['lyon_db=0.50', 'paris_db=1.00', 'paris_db=0.50', 'paris_db=4.00', 'paris_db=2.00', 'lyon_db=4.00', 'berlin_db=1.00', 'berlin_db=4.00', 'frankfurt_db=1.00', 'lyon_db=1.00', 'berlin_db=0.50', 'frankfurt_db=4.00', 'frankfurt_db=2.00', 'berlin_db=2.00', 'frankfurt_db=0.50', 'lyon_db=2.00']
16


In [2]:
list_output = []

workforce_dict = {0:5, 1:5, 2:5, 3:5, 4:5}
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

# for base_file in list_city_dt:
for base_file in ['berlin_db=1.00']:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"
    
    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.5]:
        model = 'fixed'
        dict_output = run_roster_solver_output(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        df_ = pd.DataFrame(dict_output)
        list_output.append(df_)

        model = 'flex'
        dict_output = run_roster_solver_output(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        df_ = pd.DataFrame(dict_output)
        list_output.append(df_)

        model = 'partflex'
        for max_n_shifts in range(2,5):            
            dict_output = run_roster_solver_output(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
            df_ = pd.DataFrame(dict_output)
            list_output.append(df_)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 101000 rows, 115960 columns and 465488 nonzeros
Model fingerprint: 0xcf61f473
Variable types: 99120 continuous, 16840 integer (16840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 96141 rows and 91901 columns
Presolve time: 0.18s
Presolved: 4859 rows, 24059 columns, 57814 nonzeros
Variable types: 7254 continuous, 16805 integer (16800 binary)
Found heuristic solution: objective 8676.1416667
Fou

In [3]:
df_output = pd.concat(list_output, ignore_index = True)
df_output.drop(columns = ['wage_costs','objective_value_post_wage'], inplace = True)
df_output.sort_values(by = ['city','demand_baseline','model','outsourcing_cost_multiplier','region','employee','day'], inplace = True)

df_output.to_excel(r'../code_cm/check_output_cm.xlsx', index = False)